# Context Managers

##### In Python, context managers are a way to manage resources like files, database connections, or network connections, ensuring they are properly acquired and released. This is especially useful in situations where resources need to be cleaned up after use, like closing a file or releasing a database connection.

##### Without a context manager, you’d need to use a try/finally block to ensure the file is properly closed:

In [22]:
file = open('example.txt', 'r')
try:
    content = file.read()
    print(content)
finally:
    file.close()

Hello, world!




##### The most common way to use a context manager is with the "with" statement, which allows you to encapsulate resource management code. The with statement guarantees that setup and cleanup code is always executed, regardless of whether the code block exits normally or due to an exception.

##### Context managers define two methods, \__enter__( ) and \__exit__( ), for setup and cleanup.

In [10]:
# Basic file handling with a context manager
with open("example.txt", "w") as file:
    file.write("Hello, world!")
# No need to call file.close(), it's automatically done

# Writing custom context managers in Python


#### There are two main approaches to writing custom context managers: class-based and function-based:

### Class-Based Approach

##### Here, you define a class that implements the special methods \__enter__ and \__exit__.

##### A Timer Utility example that measures execution time:

In [11]:
import time


class Timer:
   def __enter__(self):
       self.start_time = time.time()
       return self

   def __exit__(self, exc_type, exc_value, traceback):
       self.end_time = time.time()
       elapsed_time = self.end_time - self.start_time
       print(f"Elapsed time: {elapsed_time} seconds")


# Example usage
if __name__ == "__main__":
   with Timer() as timer:
       # Code block to measure the execution time
       time.sleep(2)  # Simulate some time-consuming operation

Elapsed time: 2.0050313472747803 seconds



### Function-Based Approach

##### Here, you use the context manager decorator from the contextlib module to convert any function into a context manager.

##### In this example, you write the logic for the __enter__ before the yield keyword whereas the logic for __exit__ comes after.


In [12]:
import time
from contextlib import contextmanager


@contextmanager
def timer():
   start_time = time.time()
   yield
   end_time = time.time()
   elapsed_time = end_time - start_time
   print(f"Elapsed time: {elapsed_time} seconds")


# Example usage
if __name__ == "__main__":
   with timer():
       time.sleep(2)

Elapsed time: 2.005023956298828 seconds


# Practical Examples of Context Managers

### _Managing File Operations_

In [13]:
class FileManager:
   def __init__(self, filename, mode):
       self.filename = filename
       self.mode = mode
       self.file = None

   def __enter__(self):                                 # This method is called when entering the with block
       self.file = open(self.filename, self.mode)
       return self.file

   def __exit__(self, exc_type, exc_value, traceback):  # This method is called when exiting the with block
       if self.file:
           self.file.close()


# Example usage
if __name__ == "__main__":
   with FileManager("example.txt", "w") as file:
       file.write("Hello, world!\n")

In [14]:
! cat example.txt

Hello, world!


##### Now let's modify it so that the class keeps a log of which files are opened or closed to a text file.

In [16]:
import datetime


class FileManager:
   def __init__(self, filename, mode, log_filename="file_log.txt"):
       self.filename = filename
       self.mode = mode
       self.log_filename = log_filename
       self.file = None

   def log_action(self, action):
       timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
       with open(self.log_filename, "a") as log_file:
           log_file.write(f"{timestamp} - {action}: {self.filename}\n")

   def __enter__(self):
       self.file = open(self.filename, self.mode)
       self.log_action("Opened")
       return self.file

   def __exit__(self, exc_type, exc_value, traceback):
       if self.file:
           self.file.close()
           self.log_action("Closed")


# Example Usage
with FileManager("example.txt", "r") as file:
   content = file.read()
   print(content)


Hello, world!



In [17]:
! cat file_log.txt

2024-11-07 11:32:17 - Opened: example.txt
2024-11-07 11:32:17 - Closed: example.txt
2024-11-07 11:32:38 - Opened: example.txt
2024-11-07 11:32:38 - Closed: example.txt


### _Error handling_

##### context managers in Python provide a convenient way to manage errors within resource management operations.

In [21]:
import datetime


class FileManager:
    def __init__(self, filename, mode, log_filename="file_log.txt"):
        self.filename = filename
        self.mode = mode
        self.log_filename = log_filename
        self.file = None

    def log_action(self, action):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(self.log_filename, "a") as log_file:
            log_file.write(f"{timestamp} - {action}: {self.filename}\n")

    def __enter__(self):
        self.file = open(self.filename, self.mode)
        self.log_action("Opened")
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):  # This method is modified to handle the errors
        if self.file:
            self.file.close()
            if exc_type is not None: 
                print(f"An error occurred: {exc_value}")


# Example Usage
with FileManager("example.txt", "r") as file:
    # Simulate a sample error
    content = file.read()
    print(content[150])

An error occurred: string index out of range


IndexError: string index out of range

# Conclusion

## Benefits of Python Context Manager
##### _Automatic Resource Handling_:  Context managers automatically manage resource allocation and deallocation, ensuring that resources like files, network connections, and appropriate releasing of locks after usage. This is known as automatic resource handling.
##### _Exception Safety_:  The context manager ensures that it properly cleans up the resources, preventing leaks even in the case of an error within a block.
##### _Improved Readability_:  The with statement enhances the readability and comprehension of the code by explicitly defining the scope in which the code utilizes the resource.
##### _Less Boilerplate Code_: Context managers simplify and ease the maintenance of the codebase by removing the boilerplate code required for resource management.

## Drawbacks of Python Context Manager
##### _Performance Overhead_: Using context managers, especially when creating custom ones, might have a slight overhead. However, this is generally negligible to their resource management benefits.
##### _Misuse_: Improper use of context managers can lead to unexpected behavior or bugs. For instance, if the __exit__  method does not properly handle exceptions, it might result in resource leaks.
##### _Overuse_: Overusing context managers for trivial tasks can make the code unnecessarily complex and harder to read.